### Quick and dirty frieght modelling

In [2]:
# import packages
import pandas as pd
import numpy as np
from sklearn import ensemble, model_selection, multioutput, metrics
from sklearn.externals import joblib

In [4]:
# import data
freight_fn = '../Data/Inbound_Orders_and_Live_Order_Book_for_Hackathon.xlsx'
freight_xls = pd.ExcelFile(freight_fn)

fn_hist = 'Historical Inbound Orders'
fn_live = 'Live Order Book'
hist_df = freight_xls.parse(fn_hist)
live_df = freight_xls.parse(fn_live)

In [9]:
# pivot table helper function
def get_pivot_table(in_table, val_col, index_col, cols_col, fn):
    ptable = pd.pivot_table(in_table,values=val_col, columns=cols_col, index=index_col, aggfunc=fn)
    df_table = pd.DataFrame()
    for i in index_col:
        df_table[i] = list(ptable.index.get_level_values(i))
    for i in list(ptable.columns.get_values()):
        df_table[i] = list(ptable[i])
    return df_table

In [5]:
# check data import
hist_df.head()

,Load Port,Load Port LAT,Load Port LON,Index,Eqp Type (Abrv) in Shipment,Eqp Number,ETD,Dsch Port,Dsch Port LAT,Dsch Port LON,...,PO No.,Item Class,Shipped Qty,PkgCode,Shipped Dtl Qty,PkgCode.1,Item Shipped Wgt,Unit of Measure.2,Item Shipped Msr,Unit of Measure.3
0,BANGLADESH - CHITTAGONG,22.356851,91.783182,1,20FT,1.0,2016-03-13,UK - SOUTHAMPTON,50.9097,-1.404351,...,90001.0,S-Shorts,31.0,CTN,700.0,PCS,362.90,KG,1.86,CBM
1,BANGLADESH - CHITTAGONG,22.356851,91.783182,2,20FT,1.0,2016-03-13,UK - SOUTHAMPTON,50.9097,-1.404351,...,90002.0,S-Shorts,15.0,CTN,360.0,PCS,169.50,KG,0.90,CBM
2,BANGLADESH - CHITTAGONG,22.356851,91.783182,3,20FT,1.0,2016-03-13,UK - SOUTHAMPTON,50.9097,-1.404351,...,90003.0,S-Shorts,23.0,CTN,500.0,PCS,227.40,KG,1.38,CBM
3,BANGLADESH - CHITTAGONG,22.356851,91.783182,4,20FT,1.0,2016-03-13,UK - SOUTHAMPTON,50.9097,-1.404351,...,90004.0,S-Shorts,17.0,CTN,366.0,PCS,181.85,KG,1.02,CBM
4,BANGLADESH - CHITTAGONG,22.356851,91.783182,5,20FT,1.0,2016-03-13,UK - SOUTHAMPTON,50.9097,-1.404351,...,90005.0,S-Shorts,58.0,CTN,1006.0,PCS,398.93,KG,2.53,CBM


In [7]:
# quick and dirty - drop missing data
tmp_hist_df = hist_df.dropna()
tmp_hist_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4807 entries, 0 to 5618
Data columns (total 27 columns):
Load Port                      4807 non-null object
Load Port LAT                  4807 non-null float64
Load Port LON                  4807 non-null float64
Index                          4807 non-null int64
Eqp Type (Abrv) in Shipment    4807 non-null object
Eqp Number                     4807 non-null float64
ETD                            4807 non-null datetime64[ns]
Dsch Port                      4807 non-null object
Dsch Port LAT                  4807 non-null float64
Dsch Port LON                  4807 non-null float64
ETA Dsch Port                  4807 non-null datetime64[ns]
Final Dest                     4807 non-null object
ETA Final Dest                 4807 non-null datetime64[ns]
Msr (Per Eqp)                  4807 non-null float64
Unit of Measure                4807 non-null object
Wgt (Per Eqp)                  4807 non-null float64
Unit of Measure.1              

In [8]:

tmp_hist_df['Item Class'].value_counts()

I-Knitwear       1731
B-Jeans          1098
D-Jackets         440
M-Tops            414
S-Shorts          250
F-Sweatshirts     243
Y-Skirts          234
C-Trousers        201
K-Shirts          186
A-Accessories      10
Name: Item Class, dtype: int64

In [22]:
# quick and dirty - get pivot 1
hist_pt1 = get_pivot_table(tmp_hist_df, val_col=['Shipped Qty'], index_col=['PO No.'], cols_col=['Item Class'], fn=np.sum)

In [30]:
# quick and dirty - get pivot 2
hist_pt2 = get_pivot_table(tmp_hist_df, val_col=['Item Shipped Msr'], index_col=['PO No.'], cols_col=[], fn=np.sum)

In [31]:
# join pt1 (quantity in pieces) with pt2 (volume in CBM)
hist_pt = pd.merge(hist_pt1,hist_pt2,how='inner',on=['PO No.'])

In [32]:
# fill na by zeros
hist_pt = hist_pt.fillna(0)

In [37]:
# define x y columns
mod_col_lst = list(hist_pt.columns)
x_col = []
y_col = []
for i in mod_col_lst:
    if 'Msr' not in i:
        x_col.append(i)
    else:
        y_col.append(i)

In [42]:
mod_col_lst

['PO No.',
 ('Shipped Qty', 'A-Accessories'),
 ('Shipped Qty', 'B-Jeans'),
 ('Shipped Qty', 'C-Trousers'),
 ('Shipped Qty', 'D-Jackets'),
 ('Shipped Qty', 'F-Sweatshirts'),
 ('Shipped Qty', 'I-Knitwear'),
 ('Shipped Qty', 'K-Shirts'),
 ('Shipped Qty', 'M-Tops'),
 ('Shipped Qty', 'S-Shorts'),
 ('Shipped Qty', 'Y-Skirts'),
 'Item Shipped Msr']

In [56]:
# quick and dirty - build X y
X = pd.DataFrame()
for i in x_col:
    X[i] = hist_pt[i]
y = hist_pt[y_col]

In [58]:
# baseline (multi-output)
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, train_size=0.7)

regr_multirf = multioutput.MultiOutputRegressor(ensemble.RandomForestRegressor())
regr_multirf.fit(X_train, y_train)

regr_multiada = multioutput.MultiOutputRegressor(ensemble.AdaBoostRegressor())
regr_multiada.fit(X_train, y_train)

regr_multixtr = multioutput.MultiOutputRegressor(ensemble.ExtraTreesRegressor())
regr_multixtr.fit(X_train, y_train)

regr_multigbt = multioutput.MultiOutputRegressor(ensemble.GradientBoostingRegressor())
regr_multigbt.fit(X_train, y_train)

regr_rf = ensemble.RandomForestRegressor()
regr_rf.fit(X_train, y_train)

# Predict on new data
y_multirf = regr_multirf.predict(X_test)
y_multiada = regr_multiada.predict(X_test)
y_multixtr = regr_multixtr.predict(X_test)
y_multigbt = regr_multigbt.predict(X_test)
y_rf = regr_rf.predict(X_test)

C:\Users\bryantlee\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)
C:\Users\bryantlee\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:16: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  app.launch_new_instance()


In [59]:
print(metrics.mean_squared_error(y_test, y_multirf))
print(metrics.mean_squared_error(y_test, y_multiada))
print(metrics.mean_squared_error(y_test, y_multixtr))
print(metrics.mean_squared_error(y_test, y_multigbt))
print(metrics.mean_squared_error(y_test, y_rf))

963.6190922371893
292.4292662715946
968.8917117683001
565.6494853904987
494.05943317357077


In [60]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, train_size=0.7)

regr_rfr = ensemble.RandomForestRegressor()
regr_rfr.fit(X_train, y_train)

y_regr_rfr = regr_rfr.predict(X_test)

print(metrics.mean_squared_error(y_test,y_regr_rfr))

189.95086501570918


C:\Users\bryantlee\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)
C:\Users\bryantlee\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


In [61]:
joblib.dump(regr_rfr, 'freight_pred.pkl')